# Table of GHG concentrations and ERF

In [1]:
import fair
from fair.forcing.ghg import meinshausen
import numpy as np
import scipy.stats as st
import pandas as pd

from ar6.constants.gases import radeff, is_shortlived

## Greenhouse gas concentrations

In [2]:
ghg_obs = pd.read_excel('../data_input/observations/LLGHG_history_AR6_v9_updated.xlsx', skiprows=22, sheet_name="MR_output", index_col=0)
for addyear in range(1751,1850):
    ghg_obs.loc[addyear, 'YYYY'] = np.nan
ghg_obs = ghg_obs.sort_index()
ghg_obs = ghg_obs.interpolate()
ghg_obs

,CO2,CH4,N2O,HFC-134a,HFC-23,HFC-32,HFC-125,HFC-143a,HFC-152a,HFC-227ea,...,C7F16,C8F18,CFC-112,CFC-112a,CFC-113a,CFC-114a,HCFC-133a,HCFC-31,HCFC-124,YYYY
YYYY,,,,,,,,,,,,,,,,,,,,,
1750,278.300,729.2000,270.1000,0.0000,0.000,0.000218,0.0000,0.0000,0.0000,0.000004,...,0.00,0.0,0.000,0.000,0.00,0.00,0.00000,0.000,0.00,NaN
1751,278.372,729.9840,270.1200,0.0000,0.000,0.000218,0.0000,0.0000,0.0000,0.000004,...,0.00,0.0,0.000,0.000,0.00,0.00,0.00000,0.000,0.00,NaN
1752,278.444,730.7680,270.1400,0.0000,0.000,0.000218,0.0000,0.0000,0.0000,0.000004,...,0.00,0.0,0.000,0.000,0.00,0.00,0.00000,0.000,0.00,NaN
1753,278.516,731.5520,270.1600,0.0000,0.000,0.000218,0.0000,0.0000,0.0000,0.000004,...,0.00,0.0,0.000,0.000,0.00,0.00,0.00000,0.000,0.00,NaN
1754,278.588,732.3360,270.1800,0.0000,0.000,0.000218,0.0000,0.0000,0.0000,0.000004,...,0.00,0.0,0.000,0.000,0.00,0.00,0.00000,0.000,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,399.420,1834.0055,328.3025,83.4145,28.042,9.985158,18.0910,17.5655,6.5970,1.096239,...,0.11,0.0,0.412,0.066,0.68,1.05,0.39897,0.083,1.02,NaN
2016,402.850,1842.1665,329.1240,89.4825,28.950,11.972602,20.4815,19.1555,6.6600,1.205802,...,0.11,0.0,0.412,0.067,0.68,1.04,0.42501,0.083,1.02,NaN
2017,405.000,1849.2365,330.0400,95.7485,29.975,14.153317,23.2450,20.8340,6.8175,1.327504,...,0.11,0.0,0.412,0.067,0.68,1.04,0.43989,0.083,1.02,NaN


In [3]:
forcing_minor_1750 = pd.read_csv('../data_output/AR6_ERF_minorGHGs_1750-2019.csv', index_col=0)
list_signif_0005 = ['CO2', 'CH4', 'N2O'] + forcing_minor_1750.loc[:,forcing_minor_1750.loc[2019,:]>0.0005].columns.tolist()
list_signif_0010 = ['CO2', 'CH4', 'N2O'] + forcing_minor_1750.loc[:,forcing_minor_1750.loc[2019,:]>=0.001].columns.tolist()
print(list_signif_0005)
print(list_signif_0010)

['CO2', 'CH4', 'N2O', 'HFC-134a', 'HFC-23', 'HFC-32', 'HFC-125', 'HFC-143a', 'HFC-152a', 'HFC-245fa', 'SF6', 'SO2F2', 'CF4', 'C2F6', 'c-C4F8', 'CFC-12', 'CFC-11', 'CFC-113', 'CFC-114', 'CFC-115', 'CFC-13', 'HCFC-22', 'HCFC-141b', 'HCFC-142b', 'CCl4', 'CH2Cl2', 'Halon-1211', 'Halon-1301']
['CO2', 'CH4', 'N2O', 'HFC-134a', 'HFC-23', 'HFC-32', 'HFC-125', 'HFC-143a', 'SF6', 'CF4', 'C2F6', 'CFC-12', 'CFC-11', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b', 'CCl4']


In [4]:
list_signif = [   # from table 2.3 - not the same as ERF > 0.001
    'CO2',
    'CH4',
    'N2O',
    'CFC-11',
    'CFC-12',
    'CFC-113',
    'CFC-114',
    'CFC-115',
    'CCl4',
    'Halon-1211',
    'Halon-1301',
    'HCFC-22',
    'HCFC-141b',
    'HCFC-142b',
    'HFC-23',
    'HFC-32',
    'HFC-125',
    'HFC-134a',
    'HFC-143a',
    'CF4',
    'C2F6',
    'SF6',
]

In [5]:
gases = ghg_obs.columns.to_list()
gases.remove('YYYY')

In [6]:
ghg_obs.loc[[2019, 2011, 1850, 1750],list_signif_0010].round(
    {
        'CO2':1,
        'CH4':0,
        'N2O':1,
        'CFC-11':0,
        'CFC-12':0,
        'CFC-113':1,
        'CFC-114':1,
        'CFC-115':2,
        'CCl4':1,
        'Halon-1211':2,
        'Halon-1301':2,
        'HCFC-22':0,
        'HCFC-141b':1,
        'HCFC-142b':1,
        'HFC-23':1,
        'HFC-32':1,
        'HFC-125':1,
        'HFC-134a':1,
        'HFC-143a':1,
        'CF4':1,
        'C2F6':2,
        'SF6':2,
    }
).T.style.format('{0:g}')

YYYY,2019,2011,1850,1750
CO2,409.8,390.4,285.5,278.3
CH4,1866,1803,808,729
N2O,332.1,324.4,272.1,270.1
HFC-134a,107.6,62.7,0,0
HFC-23,32.4,24.1,0,0
HFC-32,20,4.7,0,0
HFC-125,29.4,10.3,0,0
HFC-143a,24,12,0,0
SF6,9.95,7.3,0,0
CF4,85.5,79,34,34


In [7]:
forcing1750 = {}
co2base1750 = np.zeros(270)
ch4base1750 = np.zeros(270)
n2obase1750 = np.zeros(270)
forcing1850 = {}
co2base1850 = np.zeros(270)
ch4base1850 = np.zeros(270)
n2obase1850 = np.zeros(270)
c = np.array([ghg_obs['CO2'].values, ghg_obs['CH4'].values, ghg_obs['N2O'].values])
for i, year in enumerate(range(1750,2020)):
    co2base1750[i], ch4base1750[i], n2obase1750[i] = meinshausen(c[:,i], [ghg_obs.loc[1750,'CO2'], ghg_obs.loc[1750,'CH4'], ghg_obs.loc[1750,'N2O']], scale_F2x=False)
    co2base1850[i], ch4base1850[i], n2obase1850[i] = meinshausen(c[:,i], [ghg_obs.loc[1850,'CO2'], ghg_obs.loc[1850,'CH4'], ghg_obs.loc[1850,'N2O']], scale_F2x=False)

# include rapid adjustments for CO2, CH4 and N2O
forcing1750['CO2'] = 1.05 * co2base1750
forcing1750['CH4'] = 0.86 * ch4base1750
forcing1750['N2O'] = 1.07 * n2obase1750
forcing1850['CO2'] = 1.05 * co2base1850
forcing1850['CH4'] = 0.86 * ch4base1850
forcing1850['N2O'] = 1.07 * n2obase1850

In [8]:
forcing1850['CO2'][269]

2.0120938515347433

In [9]:
trop_adjustment_scale = radeff.copy()
for key in trop_adjustment_scale.keys():
    trop_adjustment_scale[key] = 1
trop_adjustment_scale['CFC-11'] = 1.13
trop_adjustment_scale['CFC-12'] = 1.12

for gas in gases[3:]:
    forcing1750[gas] = (ghg_obs.loc[:,gas] - ghg_obs.loc[1750,gas]).values * radeff[gas] * 0.001 * trop_adjustment_scale[gas]
    forcing1850[gas] = (ghg_obs.loc[:,gas] - ghg_obs.loc[1850,gas]).values * radeff[gas] * 0.001 * trop_adjustment_scale[gas]

In [10]:
forcing1750

{'CO2': array([0.00000000e+00, 1.41606910e-03, 2.83180096e-03, 4.24719576e-03,
        5.66225366e-03, 7.07697485e-03, 8.49135951e-03, 9.90540780e-03,
        1.13191199e-02, 1.27324960e-02, 1.41455362e-02, 1.55582408e-02,
        1.69706099e-02, 1.83826437e-02, 1.97943424e-02, 2.12057061e-02,
        2.26167350e-02, 2.40274293e-02, 2.54377891e-02, 2.68478147e-02,
        2.82575062e-02, 2.96668637e-02, 3.10758876e-02, 3.24845778e-02,
        3.38929346e-02, 3.53009583e-02, 3.67086488e-02, 3.81160065e-02,
        3.95230315e-02, 4.09297239e-02, 4.23360840e-02, 4.37421119e-02,
        4.51478078e-02, 4.65531718e-02, 4.79582041e-02, 4.93629050e-02,
        5.07672745e-02, 5.21713128e-02, 5.35750202e-02, 5.49783967e-02,
        5.63814426e-02, 5.77841581e-02, 5.91865432e-02, 6.05885982e-02,
        6.19903232e-02, 6.33917184e-02, 6.47927840e-02, 6.61935201e-02,
        6.75939270e-02, 6.89940048e-02, 7.03937536e-02, 7.17931736e-02,
        7.31922650e-02, 7.45910280e-02, 7.59894628e-02, 7

In [11]:
gases

['CO2',
 'CH4',
 'N2O',
 'HFC-134a',
 'HFC-23',
 'HFC-32',
 'HFC-125',
 'HFC-143a',
 'HFC-152a',
 'HFC-227ea',
 'HFC-236fa',
 'HFC-245fa',
 'HFC-365mfc',
 'HFC-43-10mee',
 'NF3',
 'SF6',
 'SO2F2',
 'CF4',
 'C2F6',
 'C3F8',
 'c-C4F8',
 'CFC-12',
 'CFC-11',
 'CFC-113',
 'CFC-114',
 'CFC-115',
 'CFC-13',
 'HCFC-22',
 'HCFC-141b',
 'HCFC-142b',
 'CH3CCl3',
 'CCl4',
 'CH3Cl',
 'CH3Br',
 'CH2Cl2',
 'CHCl3',
 'Halon-1211',
 'Halon-1301',
 'Halon-2402',
 'n-C4F10',
 'n-C5F12',
 'n-C6F14',
 'i-C6F14',
 'C7F16',
 'C8F18',
 'CFC-112',
 'CFC-112a',
 'CFC-113a',
 'CFC-114a',
 'HCFC-133a',
 'HCFC-31',
 'HCFC-124']

In [12]:
uncert = {}
uncert['CO2'] = 0.12
uncert['CH4'] = 0.20
uncert['N2O'] = 0.15
uncert['Halogens'] = 0.19

In [13]:
gases_cfcs = ['CFC-12', 'CFC-11', 'CFC-113', 'CFC-114', 'CFC-115', 'CFC-13']
gases_hcfcs= ['HCFC-22', 'HCFC-141b', 'HCFC-142b', 'HCFC-133a', 'HCFC-31', 'HCFC-124']
gases_hfcs = ['HFC-134a', 'HFC-23', 'HFC-32', 'HFC-125', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-236fa', 'HFC-245fa', 'HFC-365mfc', 'HFC-43-10mee',]
gases_halo = gases[3:]

In [14]:
forcing1750['CFCs'] = np.zeros(270)
forcing1750['HCFCs'] = np.zeros(270)
forcing1750['HFCs'] = np.zeros(270)
forcing1750['Halogens'] = np.zeros(270)
forcing1850['CFCs'] = np.zeros(270)
forcing1850['HCFCs'] = np.zeros(270)
forcing1850['HFCs'] = np.zeros(270)
forcing1850['Halogens'] = np.zeros(270)
for gas in gases_cfcs:
    forcing1750['CFCs'] = forcing1750['CFCs'] + forcing1750[gas]
    forcing1850['CFCs'] = forcing1850['CFCs'] + forcing1750[gas]
for gas in gases_hcfcs:
    forcing1750['HCFCs'] = forcing1750['HCFCs'] + forcing1750[gas]
    forcing1850['HCFCs'] = forcing1850['HCFCs'] + forcing1750[gas]
for gas in gases_hfcs:
    forcing1750['HFCs'] = forcing1750['HFCs'] + forcing1750[gas]
    forcing1850['HFCs'] = forcing1850['HFCs'] + forcing1750[gas]
for gas in gases_halo:
    forcing1750['Halogens'] = forcing1750['Halogens'] + forcing1750[gas]
    forcing1850['Halogens'] = forcing1850['Halogens'] + forcing1750[gas]
forcing1750['Total'] = forcing1750['CO2'] + forcing1750['CH4'] + forcing1750['N2O'] + forcing1750['Halogens']
forcing1850['Total'] = forcing1850['CO2'] + forcing1850['CH4'] + forcing1850['N2O'] + forcing1850['Halogens']

In [15]:
forcing1750 = pd.DataFrame(forcing1750)
forcing1850 = pd.DataFrame(forcing1850)

In [16]:
list_signif_plus = list_signif_0010 + ['CFCs', 'HCFCs', 'HFCs', 'Halogens', 'Total']

In [17]:
forcing1750.loc[[269,261],list_signif_plus].round(
    {
        'CO2': 3,
        'CH4': 3,
        'N2O': 3,
        'CFC-11': 3,
        'CFC-12': 3,
        'CFC-113': 3,
        'CFC-114': 3,
        'CFC-115': 3,
        'Halon-1211': 3,
        'Halon-1301': 3,
        'HCFC-22': 3,
        'HCFC-141b': 3,
        'HCFC-142b': 3,
        'HFC-23': 3,
        'HFC-32': 3,
        'HFC-125': 3,
        'HFC-134a': 3,
        'HFC-143a': 3,
        'HFC-152a': 3,
        'SF6': 3,
        'SO2F2': 3,
        'NF3': 3,
        'CF4': 3,
        'C2F6': 3,
        'CH3CCl3': 3,
        'CCl4': 3,
        'CFCs': 3,
        'HCFCs': 3,
        'HFCs': 3,
        'Halogens': 3,
        'Total': 3
    }
).T.style.format('{0:g}')

,269,261
CO2,2.156,1.882
CH4,0.544,0.521
N2O,0.208,0.184
HFC-134a,0.018,0.01
HFC-23,0.006,0.005
HFC-32,0.002,0.001
HFC-125,0.007,0.002
HFC-143a,0.004,0.002
SF6,0.006,0.004
CF4,0.005,0.004


In [18]:
forcing1850.loc[[269,261],list_signif_plus].round(
    {
        'CO2': 3,
        'CH4': 3,
        'N2O': 3,
        'CFC-11': 3,
        'CFC-12': 3,
        'CFC-113': 3,
        'CFC-114': 3,
        'CFC-115': 3,
        'Halon-1211': 3,
        'Halon-1301': 3,
        'HCFC-22': 3,
        'HCFC-141b': 3,
        'HCFC-142b': 3,
        'HFC-23': 3,
        'HFC-32': 3,
        'HFC-125': 3,
        'HFC-134a': 3,
        'HFC-143a': 3,
        'HFC-152a': 3,
        'SF6': 3,
        'SO2F2': 3,
        'NF3': 3,
        'CF4': 3,
        'C2F6': 3,
        'CH3CCl3': 3,
        'CCl4': 3,
        'CFCs': 3,
        'HCFCs': 3,
        'HFCs': 3,
        'Halogens': 3,
        'Total': 3,
    }
).T.style.format('{0:g}')

,269,261
CO2,2.012,1.738
CH4,0.496,0.473
N2O,0.201,0.177
HFC-134a,0.018,0.01
HFC-23,0.006,0.005
HFC-32,0.002,0.001
HFC-125,0.007,0.002
HFC-143a,0.004,0.002
SF6,0.006,0.004
CF4,0.005,0.004


In [19]:
df = pd.concat(
    (
        ghg_obs.loc[[2019, 2011, 1850, 1750],list_signif_0010].T, 
        forcing1850.loc[[269,261],list_signif_plus].T,
        forcing1750.loc[[269,261],list_signif_plus].T
    ), axis=1
)
df.columns = ['conc 2019', 'conc 2011', 'conc 1850', 'conc 1750', 'ERF 1850-2019', 'ERF 1850-2011', 'ERF 1750-2019', 'ERF 1750-2011']
df.to_csv('../data_output/table_7.5.csv')

In [20]:
# We'll assume that uncertainty in halogens is 19%

print("CO2 1850-2019", df.loc['CO2', 'ERF 1850-2019'] * uncert['CO2'])
print("CO2 1750-2019", df.loc['CO2', 'ERF 1750-2019'] * uncert['CO2'])
print("CH4 1850-2019", df.loc['CH4', 'ERF 1850-2019'] * uncert['CH4'])
print("CH4 1750-2019", df.loc['CH4', 'ERF 1750-2019'] * uncert['CH4'])
print("N2O 1850-2019", df.loc['N2O', 'ERF 1850-2019'] * uncert['N2O'])
print("N2O 1750-2019", df.loc['N2O', 'ERF 1750-2019'] * uncert['N2O'])
print("Hal 1850-2019", df.loc['Halogens', 'ERF 1850-2019'] * uncert['Halogens'])
print("Hal 1750-2019", df.loc['Halogens', 'ERF 1750-2019'] * uncert['Halogens'])
print("Sum 1850-2019", np.sqrt((df.loc['CO2', 'ERF 1850-2019'] * uncert['CO2'])**2)+
             ((df.loc['CH4', 'ERF 1850-2019'] * uncert['CH4'])**2)+
             ((df.loc['N2O', 'ERF 1850-2019'] * uncert['N2O'])**2)+
             ((df.loc['Halogens', 'ERF 1850-2019'] * uncert['Halogens'])**2)
     )
print("Sum 1750-2019", np.sqrt((df.loc['CO2', 'ERF 1750-2019'] * uncert['CO2'])**2)+
             ((df.loc['CH4', 'ERF 1750-2019'] * uncert['CH4'])**2)+
             ((df.loc['N2O', 'ERF 1750-2019'] * uncert['N2O'])**2)+
             ((df.loc['Halogens', 'ERF 1750-2019'] * uncert['Halogens'])**2)
     )

CO2 1850-2019 0.24145126218416918
CO2 1750-2019 0.2587533510208171
CH4 1850-2019 0.09929478968149963
CH4 1750-2019 0.10879659767267456
N2O 1850-2019 0.03020762303092416
N2O 1750-2019 0.031269395262946406
Hal 1850-2019 0.0775225966884088
Hal 1750-2019 0.0775225966884088
Sum 1850-2019 0.2582329709285545
Sum 1750-2019 0.277577578763391
